In [1]:
import json
import os
import pickle
import random
import sys
import pickle

from typing import Callable, Dict, List, Optional
import haiku as hk
import ase
import ase.io
import jax
import jax.numpy as jnp
import numpy as np
import optax
import yaml

from phonax.datasets import (
    datasets,
    ph_datasets,
)
from phonax.optimizer import optimizer
from phonax.energy_force_train import energy_force_train
from phonax.loss import (
    WeightedEnergyFrocesStressLoss,
    HessianLoss,
)
from phonax.nequip_model import NequIP_JAXMD_model

from phonax.utils import (
    create_directory_with_random_name,
)

from phonax.phonons_train import (
    ph_evaluate,
    hessian_train,
    two_stage_hessian_train,
)

from phonax.predictors import (
    predict_energy_forces_stress,
    predict_hessian,
)
from phonax.phonons import (
    atoms_to_ext_graph, 
    predict_gamma_spectra_filter
)

from phonax.data_utils import to_f32

from phonax.trained_models import NequIP_JAXMD_molecule_model   

jax.config.update("jax_debug_nans", True)
jax.config.update("jax_debug_infs", True)
np.set_printoptions(precision=3, suppress=True)

# Introduction

Previously, we have been making the second derivative Hessian matrix predictions given an energy model, which is trained with zeroth and first order derivative data (i.e. energy and forces).
Here in this tutorial, we will demonstrate an alternative view of the Hessian data, and use them as part of the training data (the Hessian matrix, or the eigenvalue spectrum).

As discussed in our work, this augments the training dataset in converging the energy models. Locally, the second derivative data improves the local curvatures of the energy model landscape.
To demonstrate this here, we use molecular cases as the examples. There are two parts of this tutorial, the first one focuses on using the Hessian matrix as an extended data type to train the energy model/functional. The second part uses the eigenvalues of the Hessian matrix instead. The implications are the connections to the experimental probes for the vibrational modes such as IR / Raman / inelastic neutron scatterings.
This enables the energy model training or fine-tuning with realistic experimental data.

## Energy model training with molecular Hessian data

This section focuses on using the Hessian matrix as an extended data type to train the energy model/functional for the interatomic potentials.
The training procedure can be viewed as an extension to the conventional interatomic potential training using only the zeroth and first order training data for the energy landscape (i.e. energy and force).

For the specific example, we consdier the propane molecules as discussed in our work.
The structure of a propane molecule is mainly defined by the two torsion angles $\phi_1, \phi_2$ for the two C-C bonds.
Due to the symmetry, we only need to consider the range $[0,2\pi/3]$ for $\phi_i$.
In the valid dataset for the forces (in deriving the force MAE), we have scanned the $\phi_1, \phi_2$ angles ($31 \times 31$ grid). At each torsion angle combination, we also add a small Gaussian noise to the atomic coordinates.

In the training dataset, we only use four randomly generated propane structures (randomized torsion angles and perturbed atomic coordinates).
For the conventional training procedure, only the energy / force data is used, while the full molecular Hessians are added in the new training procedure to utilize the second order derivative information.
Below, we give more details for setting up these trainings.




### Initialize the training procedures

Here we load the training configurations and datasets, initialize the model, loss functions, and the predictors used for the next training steps.

In [2]:
# Load the config files
with open('data/propane/mol-propane.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
        
if 'save_dir_name' in config:
    save_dir_name = config['save_dir_name']
else:
    save_dir_name = create_directory_with_random_name()
print(save_dir_name)
        
# Save config 
with open(f"{save_dir_name}/config.yaml", "w") as f:
    yaml.dump(config, f)
    
# Load the energy/force training/valid dataset
EF_train_loader, EF_valid_loader, EF_test_loader, EF_r_max = datasets(
    r_max = config["cutoff"],
    config_dataset = config["dataset"],
)
#print(len(EF_train_loader.graphs),len(EF_valid_loader.graphs))

# Load the Hessian training/valid dataset
ph_train_loader, ph_valid_loader, ph_test_loader, ph_r_max, num_message_passing = ph_datasets(
    r_max = config["cutoff"],
    config_dataset = config["ph_dataset"],
    num_message_passing = config['model']['num_layers'],
)
#print(len(ph_train_loader.graphs),len(ph_valid_loader.graphs))

assert ph_r_max==EF_r_max
r_max = EF_r_max

# Initialize the NequIP model
model_fn, params, num_message_passing = NequIP_JAXMD_model(
    r_max=r_max,
    atomic_energies_dict={},
    train_graphs=EF_train_loader.graphs,
    initialize_seed=config["model"]["seed"],
    num_species = config["model"]["num_species"],
    use_sc = True,
    graph_net_steps = config["model"]["num_layers"],
    hidden_irreps = config["model"]["internal_irreps"],
    nonlinearities =  {'e': 'swish', 'o': 'tanh'},
    save_dir_name = save_dir_name,
    reload = config["initialization"]['reload'] if 'reload' in config["initialization"] else None,
)
    
print("num_params:", sum(p.size for p in jax.tree_util.tree_leaves(params)))

# Define Loss functions for energy/force and Hessians
EF_loss_fn = WeightedEnergyFrocesStressLoss(
    energy_weight = config["training"]["energy_weight"],
    forces_weight = config["training"]["forces_weight"],
    stress_weight = config["training"]["stress_weight"],
)

H_loss_fn = HessianLoss()

# Predictors for the energy/force/stress and Hessians
EF_predictor = jax.jit(
    lambda w, g: predict_energy_forces_stress(lambda *x: model_fn(w, *x), g)
)

H_predictor = jax.jit(
    lambda w, g: predict_hessian(lambda *x: model_fn(w, *x), g)
)


2024-01-11-21:14-xenophobic-aridatha
2024-01-11-21:14-xenophobic-aridatha
nums check 256 2048 10
Loaded 40 training configurations from 'data/propane/propane_force_N4.xyz'
Loaded 9610 validation configurations from 'data/propane/propane_v2_mesh_PBE_force.xyz'
Total number of configurations: train=40, valid=9610, test=0


100%|████████████████████████████████████| 9610/9610 [00:00<00:00, 24722.09it/s]
0it [00:00, ?it/s]
Pad valid: 100%|███████████████████████████| 1000/1000 [00:03<00:00, 272.31it/s]


Total number of configurations: train=4356, valid=1000, test=0
Compute the average number of neighbors: 10.000
Do not normalize the radial basis (avg_r_min=None)
Computed average Atomic Energies using least squares: {1: -6.220972980547945, 6: -2.3328648677054784}
Create NequIP (JAX-MD version) with parameters {'use_sc': True, 'graph_net_steps': 2, 'hidden_irreps': '24x0e+24x0o+12x1e+12x1o+8x2e+8x2o', 'nonlinearities': {'e': 'swish', 'o': 'tanh'}, 'r_max': 5.0, 'avg_num_neighbors': 10.0, 'avg_r_min': None, 'num_species': 100, 'radial_basis': <function bessel_basis at 0x7fb719c41080>, 'radial_envelope': <function soft_envelope at 0x7fb719c40fe0>}
num_params: 105836


### Training with energy / force training data

In this section, we demonstrate the energy model training by using only the energy and force training data, as is done in the conventional interatomic potential model training procedure.
Note in this simple propane molecule example, we only use four molecular configurations in the training dataset.



In [3]:
# force only training

Fonly_params = params.copy()

gradient_transform, steps_per_interval, max_num_intervals = optimizer(
    lr = config["training"]["learning_rate"],
    max_num_intervals = config["training"]["max_num_intervals"],
    steps_per_interval = config["training"]["steps_per_interval"],
    # weight_decay = config["training"]["weight_decay"],
)
optimizer_state = gradient_transform.init(params)
print("optimizer num_params:", sum(p.size for p in jax.tree_util.tree_leaves(optimizer_state)))
    
train_out = energy_force_train(
    EF_predictor,
    Fonly_params,
    optimizer_state,
    EF_train_loader,
    EF_valid_loader,
    EF_test_loader,
    gradient_transform,
    EF_loss_fn,
    max_num_intervals,
    steps_per_interval,
    save_dir_name,
    ema_decay = config["training"]["ema_decay"],
    patience = config["training"]["patience"],
)



optimizer num_params: 317510
Started training


eval_train:   0%|                                         | 0/4 [00:00<?, ?it/s]

Compiled function `model` for args:
cache size: 1


eval_train: 100%|███████████████████████████| 4/4 [00:05<00:00,  1.35s/it, n=36]


Interval 0: eval_train: loss=44.3532, mae_e_per_atom=750.6 meV, mae_f=522.2 meV/Å, mae_s=0.8 meV/Å³


...

Interval 19: eval_valid: loss=1.0751, mae_e_per_atom=1.1 meV, mae_f=56.3 meV/Å, mae_s=1.4 meV/Å³


eval_train: 100%|███████████████████████████| 4/4 [00:00<00:00, 65.39it/s, n=36]


Interval 20: eval_train: loss=0.0005, mae_e_per_atom=0.1 meV, mae_f=2.1 meV/Å, mae_s=1.2 meV/Å³


eval_valid: 100%|███████████████████| 1068/1068 [00:15<00:00, 68.39it/s, n=9610]


Interval 20: eval_valid: loss=1.0961, mae_e_per_atom=1.2 meV, mae_f=56.6 meV/Å, mae_s=1.4 meV/Å³
Training complete


### Training with molecular Hessians

In the next section, we show the energy model training with additional second order Hessian data beynd the force data at the first order derivative.
Specifically, there are two tages in the training procedure.
The first stage, or the warm-up stage, uses only the energy and force data to initialize the energy model training.
After trained with several warm-up steps, the second training stage adds the Hessian data in computing the loss function.
At the second stage, the update gradients come from both the energy / force training losses, and the Hessian losses, and we combine the two (weighted sum) to get the overall training gradient.

From the training results below, one can see the valid force MAE further decreases once we have included the Hessian training data in the second stage.

We note that, it remains an open question to find the optimal training procedure in adding the second order Hessian data. For example, one can perform the training without warm-up steps, or gradually ramp up the training weights associated with the Hessian losses.
It would be relevant to examine the energy model converged for their predictions and stability under molecular dynamics simulations.


In [3]:
FH_params = params.copy()

# restart the optimizer state
gradient_transform, steps_per_interval, max_num_intervals = optimizer(
    lr = config["training"]["learning_rate"],
    max_num_intervals = config["training"]["max_num_intervals"],
    steps_per_interval = config["training"]["steps_per_interval"],
)
optimizer_state = gradient_transform.init(params)
print("optimizer num_params:", sum(p.size for p in jax.tree_util.tree_leaves(optimizer_state)))

two_stage_hessian_train(
    energy_forces_stress_predictor=EF_predictor,
    phonon_predictor=H_predictor,
    params=FH_params,
    gradient_transform=gradient_transform,
    optimizer_state=optimizer_state,
    steps_per_interval=steps_per_interval,
    EF_loss_fn=EF_loss_fn,
    H_loss_fn=H_loss_fn,
    EF_train_loader=EF_train_loader,
    EF_valid_loader=EF_valid_loader,
    H_train_loader=ph_train_loader,
    H_valid_loader=ph_valid_loader,
    warmup_num_intervals=config['training']['warmup_num_intervals'],
    phtrain_num_intervals=config['training']['phtrain_num_intervals'],
)




optimizer num_params: 317510


eval_train:   0%|                                         | 0/4 [00:00<?, ?it/s]

Compiled function `model` for args:
cache size: 1


eval_train: 100%|███████████████████████████| 4/4 [00:05<00:00,  1.37s/it, n=36]


Interval 0: eval_train: loss=44.3532, mae_e_per_atom=750.565 meV, mae_f=522.178 meV/Å, mae_s=0.802 meV/Å³


...

Evaluating hessian train: 100%|█| 121/121 [00:00<00:00, 145.88it/s, n_graphs=435


interval 14: hessian train hessian MAE 0.040 eV/A2 


Evaluating hessian valid: 100%|█| 28/28 [00:00<00:00, 149.89it/s, n_graphs=1000]


interval 14: hessian valid hessian MAE 0.037 eV/A2 


Interval 14: 100%|███████████████| 500/500 [00:10<00:00, 48.39it/s, loss=0.0194]


# Energy model fine-tuning with Hessian spectrum data




Here in this section,

In [6]:
model_fn, params, num_message_passing, r_max = NequIP_JAXMD_molecule_model(os.path.join(os.getcwd(), 'trained-models'))

train_crystal = ase.io.read('data/molecules/benzene.vasp')
train_crystal.pbc=False
train_graph = atoms_to_ext_graph(train_crystal, r_max, num_message_passing=1)
train_graph = jax.tree_util.tree_map(to_f32, train_graph)

test_crystal = ase.io.read('data/molecules/toluene.vasp')
test_crystal.pbc=False
test_graph = atoms_to_ext_graph(test_crystal, r_max, num_message_passing=1)
test_graph = jax.tree_util.tree_map(to_f32, test_graph)

train_masses = ase.data.atomic_masses[train_graph.nodes.species][train_graph.nodes.mask_primitive]
test_masses = ase.data.atomic_masses[test_graph.nodes.species][test_graph.nodes.mask_primitive]

def read_gamma_modes(fname):
    all_modes = []
    with open(fname,'r') as f:
        for line in f:
            if "Mode" in line:
                all_modes.append(float(line.strip().split(',')[-1].strip().split()[0]))
    all_modes = jnp.array(all_modes)
    return all_modes
benzene_gamma = read_gamma_modes('data/molecules/benzene_gamma')
toluene_gamma = read_gamma_modes('data/molecules/toluene_gamma')


def loss_fn_train(spectra_in):
    return jnp.mean(jnp.abs(spectra_in-benzene_gamma))
def loss_fn_test(spectra_in):
    return jnp.mean(jnp.abs(spectra_in-toluene_gamma))
def mae_spectra(pred_spectra,target_spectra):
    return jnp.mean(jnp.abs(pred_spectra-target_spectra))

gradient_transform = optax.adam(5E-5)

optimizer_state = gradient_transform.init(params)

spectra_predictor = jax.jit(
    lambda w: predict_gamma_spectra_filter(w,model_fn,train_graph,train_masses)  #predict_gamma_spectra(lambda *x: model_fn(w, *x), test_graph)
)

valid_modes_original = predict_gamma_spectra_filter(params,model_fn,test_graph,test_masses)


Create NequIP (JAX-MD version) with parameters {'use_sc': True, 'graph_net_steps': 2, 'hidden_irreps': '96x0e+ 96x0o  + 64x1e +64x1o +64x2e +64x2o', 'nonlinearities': {'e': 'swish', 'o': 'tanh'}, 'r_max': 5.0, 'avg_num_neighbors': 53.2808396682792, 'avg_r_min': None, 'num_species': 100, 'radial_basis': <function bessel_basis at 0x7f77bd701440>, 'radial_envelope': <function soft_envelope at 0x7f77bd7013a0>}


In [7]:
@jax.jit
def update_fn(
    params, optimizer_state
):
    loss, grad = jax.value_and_grad(
        lambda params: loss_fn_train(spectra_predictor(params))
    )(params)

    updates, optimizer_state = gradient_transform.update(
        grad, optimizer_state, params
    )
    params = optax.apply_updates(params, updates)
    return loss, params, optimizer_state

spectra_mae = []

for ind in range(200):
    loss, params, optimizer_state = update_fn(params, optimizer_state)
    train_modes = predict_gamma_spectra_filter(params,model_fn,train_graph,train_masses)
    valid_modes = predict_gamma_spectra_filter(params,model_fn,test_graph,test_masses)
    train_mae = mae_spectra(train_modes,benzene_gamma)
    valid_mae = mae_spectra(valid_modes,toluene_gamma)

    print(
        'Benzene spectra MAE: ', 
        train_mae,
        'Toluene spectra MAE: ',
        valid_mae,
    )
    spectra_mae.append([train_mae, valid_mae])


Benzene spectra MAE:  88.0602 Toluene spectra MAE:  74.18877
Benzene spectra MAE:  85.86992 Toluene spectra MAE:  72.87502
Benzene spectra MAE:  83.41573 Toluene spectra MAE:  70.76455
Benzene spectra MAE:  80.89786 Toluene spectra MAE:  68.3744
Benzene spectra MAE:  78.783806 Toluene spectra MAE:  65.71441
Benzene spectra MAE:  76.697945 Toluene spectra MAE:  63.42712
Benzene spectra MAE:  74.64182 Toluene spectra MAE:  61.302
Benzene spectra MAE:  72.42226 Toluene spectra MAE:  59.635105
Benzene spectra MAE:  70.277 Toluene spectra MAE:  58.176296
Benzene spectra MAE:  68.560936 Toluene spectra MAE:  56.826687
Benzene spectra MAE:  67.40696 Toluene spectra MAE:  55.650597
Benzene spectra MAE:  66.224625 Toluene spectra MAE:  54.374023
Benzene spectra MAE:  65.042625 Toluene spectra MAE:  53.331573
Benzene spectra MAE:  64.294586 Toluene spectra MAE:  52.429073
Benzene spectra MAE:  63.428223 Toluene spectra MAE:  51.64859
Benzene spectra MAE:  62.439117 Toluene spectra MAE:  50.93556